In [1]:
%pip install mne==0.23.4

  Using cached mne-0.23.4-py3-none-any.whl (6.9 MB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install pyedflib

Note: you may need to restart the kernel to use updated packages.  Downloading pyEDFlib-0.1.22-cp36-cp36m-win_amd64.whl (1.1 MB)



In [6]:
import os
import pandas as pd
import glob
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import filtfilt
from scipy import stats
import scipy
import mne
import pyedflib

np.random.seed(0)

In [8]:
file_choices = np.random.choice(os.listdir('./data/eeg_fpz_cz'), size=10, replace=False)
file_choices = os.listdir('./data/eeg_fpz_cz')
file_choices

['SC4001E0.npz',
 'SC4002E0.npz',
 'SC4011E0.npz',
 'SC4012E0.npz',
 'SC4021E0.npz',
 'SC4022E0.npz',
 'SC4031E0.npz',
 'SC4032E0.npz',
 'SC4041E0.npz',
 'SC4042E0.npz',
 'SC4051E0.npz',
 'SC4052E0.npz',
 'SC4061E0.npz',
 'SC4062E0.npz',
 'SC4071E0.npz',
 'SC4072E0.npz',
 'SC4081E0.npz',
 'SC4082E0.npz',
 'SC4091E0.npz',
 'SC4092E0.npz',
 'SC4101E0.npz',
 'SC4102E0.npz',
 'SC4111E0.npz',
 'SC4112E0.npz',
 'SC4121E0.npz',
 'SC4122E0.npz',
 'SC4131E0.npz',
 'SC4141E0.npz',
 'SC4142E0.npz',
 'SC4151E0.npz',
 'SC4152E0.npz',
 'SC4161E0.npz',
 'SC4162E0.npz',
 'SC4171E0.npz',
 'SC4172E0.npz',
 'SC4181E0.npz',
 'SC4182E0.npz',
 'SC4191E0.npz',
 'SC4192E0.npz',
 'SC4201E0.npz',
 'SC4202E0.npz',
 'SC4211E0.npz',
 'SC4212E0.npz',
 'SC4221E0.npz',
 'SC4222E0.npz',
 'SC4231E0.npz',
 'SC4232E0.npz',
 'SC4241E0.npz',
 'SC4242E0.npz',
 'SC4251E0.npz',
 'SC4252E0.npz',
 'SC4261F0.npz',
 'SC4262F0.npz',
 'SC4271F0.npz',
 'SC4272F0.npz',
 'SC4281G0.npz',
 'SC4282G0.npz',
 'SC4291G0.npz',
 'SC4292G0.npz

# Reading A File

In [9]:
file = np.load('./data/eeg_fpz_cz/SC4352F0.npz', allow_pickle=True)
file.files

['x', 'y', 'fs', 'ch_label', 'header_raw', 'header_annotation']

In [10]:
# there are 6 arrays, reading X

X = file['x'].squeeze()

# Making pandas DataFrame for esay understanding
pd.DataFrame(X).head()

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,1.806593,-0.446154,-0.626374,-6.393406,-1.527472,0.454945,-3.329670,-6.303297,-2.879121,-1.347253,...,-7.384615,-11.169230,-8.015385,1.085714,-5.582417,-12.250549,-11.439561,3.158242,2.167033,-0.085714
1,-1.707692,-3.780220,-5.852747,-12.971429,0.184615,2.797802,-1.617582,-1.707692,0.454945,3.428571,...,12.439561,9.105494,6.852747,3.698901,4.239561,0.454945,-7.835165,-4.951648,6.942857,7.573627
2,9.465934,9.736263,19.468132,17.756044,19.738462,3.789011,-0.896703,-1.797802,-2.698901,5.681319,...,-11.800000,-3.780220,-12.520879,-12.070330,-18.468132,-12.160439,-14.773626,-8.556044,-11.349450,-11.079121
3,-9.727472,-6.213187,-9.276923,7.123077,-5.131868,-20.810989,-8.105494,-14.232967,-20.720879,-21.621979,...,-15.134066,1.626374,12.980220,12.709890,8.204395,4.690110,3.158242,0.815385,-1.437363,2.527472
4,2.076923,3.338462,4.600000,3.969231,2.527472,2.257143,5.230769,2.076923,4.870330,-0.896703,...,-0.536264,4.780220,-6.483517,-13.602198,-6.213187,-5.672527,-10.898901,-21.441759,-16.125275,-11.980220


In [11]:
# ploting first row and first 500 points

plt.figure(figsize=(20,10))
plt.plot(X[0][:500])

AttributeError: module 'matplotlib.colors' has no attribute 'to_rgba'

# Fast Fourier Transformation

In [13]:
def dft(x):
    x = np.asarray(x, dtype=float)
    N = x.shape[0]
    n = np.arange(N)
    k = n.reshape((N, 1))
    M = np.exp(-2j * np.pi * k * n / N)
    return np.dot(M, x)

In [14]:
def fft(x):
    x = np.asarray(x, dtype=float)
    N = x.shape[0]
    if N % 2 > 0:
        raise ValueError("must be a power of 2")
    elif N <= 2:
        return dft(x)
    else:
        X_even = fft(x[::2])
        X_odd = fft(x[1::2])
        terms = np.exp(-2j * np.pi * np.arange(N) / N)
        return np.concatenate([X_even + terms[:int(N/2)] * X_odd,
                               X_even + terms[int(N/2):] * X_odd])

In [15]:
def fft_v(x):
    x = np.asarray(x, dtype=float)
    x = x.reshape((1, -1))
    N = x.shape[0]
    if np.log2(N) % 1 > 0:
        raise ValueError("must be a power of 2")
        
    N_min = min(N, 2)
    
    n = np.arange(N_min)
    k = n[:, None]
    M = np.exp(-2j * np.pi * n * k / N_min)
    X = np.dot(M, x.reshape((N_min, -1)))
    
    while X.shape[0] < N:
            X_even = X[:, :int(X.shape[1] / 2)]
            X_odd = X[:, int(X.shape[1] / 2):]
            terms = np.exp(-1j * np.pi * np.arange(X.shape[0])
                            / X.shape[0])[:, None]
            X = np.vstack([X_even + terms * X_odd,
                           X_even - terms * X_odd])
    return X.ravel().astype('complex_')

# BandPass Filtering

In [16]:
def bandPassFilter(raw, raw_signals):
    
    fs = 100.0
    highcut = float(raw_signals._raw_extras[0]['lowpass'][0]) * 0.25
    lowcut = float(raw_signals._raw_extras[0]['highpass'][0]) * 0.25
    
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    
    order = 2
    
    b, a = scipy.signal.butter(order, [low, high], 'bandpass', analog=False)
    
    y = scipy.signal.filtfilt(b, a, raw)
    
    return y

# Reading the Original Data

In [17]:

def read_original_data(code):
    file_name_signal = f'./sleep-edf-database-expanded-1.0.0/sleep-cassette/{code}-PSG.edf'
    
    f = pyedflib.EdfReader(file_name_signal)
    signal_labels = f.getSignalLabels()
    
    raw_signal = mne.io.read_raw_edf(file_name_signal, eog=None, misc=None, stim_channel='auto', preload=True, verbose=None)
    return raw_signal

# Compiling everything

In [18]:

for file_code in file_choices:
    
    file = np.load(f'./data/eeg_fpz_cz/{file_code}', allow_pickle=True)
    X = file['x'].squeeze()
    orignal_signal = read_original_data(file_code.replace('.npz', ''))
    X_new = np.array(file['x'])
    
    for idx, row in enumerate(X):
        fft_X = fft_v(row)
        BandPass_X = bandPassFilter(fft_X,orignal_signal)
        X_new[idx] = BandPass_X.reshape(-1, 1)

    np.savez(f'./Data with pre-processing/{file_code}',x=X_new, y=file['y'], fs=file['fs'],
             ch_label=file['ch_label'], header_raw=file['header_raw'], header_annotation=file['header_annotation'], allow_pickle=True)
    print(f"{file_code} is saved")
    print("============"*5)

Extracting EDF parameters from C:\Users\arunk\Downloads\sleep-stage-data\content\sleep-edf-database-expanded-1.0.0\sleep-cassette\SC4001E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 7949999  =      0.000 ... 79499.990 secs...


C:\Users\arunk\anaconda\envs\sleep\lib\site-packages\ipykernel_launcher.py:11: ComplexWarning: Casting complex values to real discards the imaginary part
  # This is added back by InteractiveShellApp.init_path()


SC4001E0.npz is saved
Extracting EDF parameters from C:\Users\arunk\Downloads\sleep-stage-data\content\sleep-edf-database-expanded-1.0.0\sleep-cassette\SC4002E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 8489999  =      0.000 ... 84899.990 secs...
SC4002E0.npz is saved
Extracting EDF parameters from C:\Users\arunk\Downloads\sleep-stage-data\content\sleep-edf-database-expanded-1.0.0\sleep-cassette\SC4011E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 8405999  =      0.000 ... 84059.990 secs...
SC4011E0.npz is saved
Extracting EDF parameters from C:\Users\arunk\Downloads\sleep-stage-data\content\sleep-edf-database-expanded-1.0.0\sleep-cassette\SC4012E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 8549999  =      0.000 ... 85499.990 secs...
SC4012E0.npz is saved
Extracting EDF parameters from C:\Users

SC4111E0.npz is saved
Extracting EDF parameters from C:\Users\arunk\Downloads\sleep-stage-data\content\sleep-edf-database-expanded-1.0.0\sleep-cassette\SC4112E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 8339999  =      0.000 ... 83399.990 secs...
SC4112E0.npz is saved
Extracting EDF parameters from C:\Users\arunk\Downloads\sleep-stage-data\content\sleep-edf-database-expanded-1.0.0\sleep-cassette\SC4121E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 8357999  =      0.000 ... 83579.990 secs...
SC4121E0.npz is saved
Extracting EDF parameters from C:\Users\arunk\Downloads\sleep-stage-data\content\sleep-edf-database-expanded-1.0.0\sleep-cassette\SC4122E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 7817999  =      0.000 ... 78179.990 secs...
SC4122E0.npz is saved
Extracting EDF parameters from C:\Users

SC4222E0.npz is saved
Extracting EDF parameters from C:\Users\arunk\Downloads\sleep-stage-data\content\sleep-edf-database-expanded-1.0.0\sleep-cassette\SC4231E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 8237999  =      0.000 ... 82379.990 secs...
SC4231E0.npz is saved
Extracting EDF parameters from C:\Users\arunk\Downloads\sleep-stage-data\content\sleep-edf-database-expanded-1.0.0\sleep-cassette\SC4232E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 7919999  =      0.000 ... 79199.990 secs...
SC4232E0.npz is saved
Extracting EDF parameters from C:\Users\arunk\Downloads\sleep-stage-data\content\sleep-edf-database-expanded-1.0.0\sleep-cassette\SC4241E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 8105999  =      0.000 ... 81059.990 secs...
SC4241E0.npz is saved
Extracting EDF parameters from C:\Users

SC4332F0.npz is saved
Extracting EDF parameters from C:\Users\arunk\Downloads\sleep-stage-data\content\sleep-edf-database-expanded-1.0.0\sleep-cassette\SC4341F0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 8249999  =      0.000 ... 82499.990 secs...
SC4341F0.npz is saved
Extracting EDF parameters from C:\Users\arunk\Downloads\sleep-stage-data\content\sleep-edf-database-expanded-1.0.0\sleep-cassette\SC4342F0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 8369999  =      0.000 ... 83699.990 secs...
SC4342F0.npz is saved
Extracting EDF parameters from C:\Users\arunk\Downloads\sleep-stage-data\content\sleep-edf-database-expanded-1.0.0\sleep-cassette\SC4351F0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 8129999  =      0.000 ... 81299.990 secs...
SC4351F0.npz is saved
Extracting EDF parameters from C:\Users

In [24]:
# # Remove non-mat files, and perform ascending sort
# allfiles = os.listdir('./Data with pre-processing')
# npzfiles = []
# for idx, f in enumerate(allfiles):
#     if ".npz" in f:
#         npzfiles.append(os.path.join('./Data with pre-processing', f))
# npzfiles.sort()

# if 10 is not None:
#     npzfiles = npzfiles[:10]

# subject_files = []
# for idx, f in enumerate(allfiles):
#     if 1 < 10:
#         pattern = re.compile("[a-zA-Z0-9]*0{}[1-9]E0\.npz$".format(1))
#     else:
#         pattern = re.compile("[a-zA-Z0-9]*{}[1-9]E0\.npz$".format(1))
#     if pattern.match(f):
#         subject_files.append(os.path.join('./Data with pre-processing', f))

# if len(subject_files) == 0:
#     for idx, f in enumerate(allfiles):
#         if 1 < 10:
#             pattern = re.compile("[a-zA-Z0-9]*0{}[1-9]J0\.npz$".format(1))
#         else:
#             pattern = re.compile("[a-zA-Z0-9]*{}[1-9]J0\.npz$".format(1))
#         if pattern.match(f):
#             subject_files.append(os.path.join('./Data with pre-processing', f))

# print(subject_files)
# train_files = list(set(npzfiles) - set(subject_files))
# train_files.sort()
# train_files=train_files[:10]
# subject_files.sort()
# subject_files=subject_files[:10]
# print(train_files)

['./Data with pre-processing\\SC4011E0.npz', './Data with pre-processing\\SC4012E0.npz']
['./Data with pre-processing\\SC4001E0.npz', './Data with pre-processing\\SC4002E0.npz', './Data with pre-processing\\SC4021E0.npz', './Data with pre-processing\\SC4022E0.npz', './Data with pre-processing\\SC4031E0.npz', './Data with pre-processing\\SC4032E0.npz', './Data with pre-processing\\SC4041E0.npz', './Data with pre-processing\\SC4042E0.npz']


In [23]:
# import re

In [25]:
# subject_files

['./Data with pre-processing\\SC4011E0.npz',
 './Data with pre-processing\\SC4012E0.npz']

In [26]:
# train_files

['./Data with pre-processing\\SC4001E0.npz',
 './Data with pre-processing\\SC4002E0.npz',
 './Data with pre-processing\\SC4021E0.npz',
 './Data with pre-processing\\SC4022E0.npz',
 './Data with pre-processing\\SC4031E0.npz',
 './Data with pre-processing\\SC4032E0.npz',
 './Data with pre-processing\\SC4041E0.npz',
 './Data with pre-processing\\SC4042E0.npz']

In [28]:
# # a = []
# for sx in train_files:
# #     a.append(sx.split('\\')[-1])
# print(a)

['SC4011E0.npz', 'SC4012E0.npz', 'SC4001E0.npz', 'SC4002E0.npz', 'SC4021E0.npz', 'SC4022E0.npz', 'SC4031E0.npz', 'SC4032E0.npz', 'SC4041E0.npz', 'SC4042E0.npz']


In [32]:
/

'SC4011E0.npz" "SC4012E0.npz" "SC4001E0.npz" "SC4002E0.npz" "SC4021E0.npz" "SC4022E0.npz" "SC4031E0.npz" "SC4032E0.npz" "SC4041E0.npz" "SC4042E0.npz'

In [ ]:
# "SC4011E0.npz" "SC4012E0.npz" "SC4001E0.npz" "SC4002E0.npz" "SC4021E0.npz" "SC4022E0.npz" "SC4031E0.npz" "SC4032E0.npz" "SC4041E0.npz" "SC4042E0.npz"